# Clasificación de textos

En esta clase, exploraremos datos de mensajes de texto y crearemos modelos para predecir si un mensaje es spam o no.

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Pregunta 1
¿Qué porcentaje de los documentos en `spam_data` son spam?

*Esta función debe devolver un valor flotante, el valor porcentual (es decir, $ ratio * 100 $).*

In [3]:
def respuesta_uno():
    spam_data['target'].mean()
    spam=spam_data[spam_data['target'] != 0]
    print(len(spam), len(spam_data))
    return spam_data['target'].mean()*100

In [4]:
respuesta_uno()

747 5572


13.406317300789663

### Pregunta 2

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial. Calcule medidas de exactitud, presición, recall y f1-score usando los datos de prueba transformados.

*Esta función debe devolver las cuatro medidas de evaluación como una lista con los 4 valores en el orden solicitado cada valor como flotante.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def respuesta_dos():
    scores=[]
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    
    clf=MultinomialNB()
    clf.fit(X_train_vectorized, y_train)
    
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    
    scores.append(accuracy_score(y_test,predictions))
    scores.append(precision_score(y_test,predictions))
    scores.append(recall_score(y_test,predictions))
    scores.append(f1_score(y_test,predictions))
    
    spam=predictions[predictions!= 0]
    print('Predicciones Spam:',len(spam))
    print('Porcentaje Spam:',len(spam)/len(predictions)*100)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(confusion_matrix(y_test, predictions))
    print(tn, fp, fn, tp)
    return scores

In [6]:
respuesta_dos()

Predicciones Spam: 184
Porcentaje Spam: 13.208901651112706
[[1193    3]
 [  16  181]]
1193 3 16 181


[0.9863603732950467,
 0.9836956521739131,
 0.9187817258883249,
 0.9501312335958005]

### Pregunta 3

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial con suavizado (smoothing) `alpha = 0.1`. Encuentre el área bajo la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def respuesta_tres():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clf=MultinomialNB()
    clf.fit(X_train_vectorized, y_train)
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [8]:
respuesta_tres()

[0 0 0 ... 0 0 1]


0.9581366823421557

### Pregunta 4

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **3**.

Luego, ajuste un modelo de clasificador Naive Bayes multinomial con suavizado (smoothing) `alfa = 0.1` y calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [9]:
def respuesta_cuatro():
    vectorizer = TfidfVectorizer(min_df=0.3)
    X_train_vect = vectorizer.fit_transform(X_train)
    clf = MultinomialNB()
    clf.fit(X_train_vect, y_train)
    X_test_vect = vectorizer.transform(X_test)
    predictions = clf.predict(X_test_vect)
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [10]:
respuesta_cuatro()

[0 0 0 ... 0 0 0]


0.5

### Pregunta 5

¿Cuál es la longitud promedio de los documentos (número de caracteres) para los documentos spam y no spam?

*Esta función debe devolver una tupla (longitud promedio no spam, longitud promedio de spam).*

In [11]:
def respuesta_cinco():
    spam=spam_data[spam_data['target'] != 0]
    sumLon = 0
    for row in spam.iterrows():
        sumLon += len(row[1].array[0])
    promSpam = sumLon/len(spam)
    no_spam= spam_data[spam_data['target'] != 1]
    sum1 = 0
    for row in no_spam.iterrows():
        sum1 += len(row[1].array[0])
    promNoSpam = sum1/len(no_spam)
    return (promNoSpam,promSpam)

In [12]:
respuesta_cinco()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [13]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Pregunta 6

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5**.

Usando esta matriz de término de documento y una característica adicional, **la longitud del documento (número de caracteres)**, ajustar a un modelo de Clasificación de Vector de Soporte con regularización `C = 10000`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [14]:
from sklearn.svm import SVC

def aux(data):
    longitud = [] 
    for row in data:
        longitud.append(len(row))
    return np.resize(np.array(longitud),(len(longitud),1))
    

def respuesta_seis():
    long = aux(X_train)
    vectorizer = TfidfVectorizer(max_df=0.5)
    X_train_vect = vectorizer.fit_transform(X_train)
    X_train_array = X_train_vect.toarray()
    X_train_array = np.concatenate((X_train_array,long),axis =1)
    clf = SVC(C = 10000)
    clf.fit(X_train_array,np.array(y_train))
    print(clf)
    long1= aux(X_test)
    X_test_vect = vectorizer.transform(X_test)
    X_test_arra= np.concatenate((X_test_vect.toarray(),long1), axis=1)
    predictions = clf.predict(X_test_arra)
    print(predictions)
    return roc_auc_score(y_test,predictions)


In [15]:
respuesta_seis()

SVC(C=10000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[0 0 0 ... 0 0 1]


0.9665870159414631

### Pregunta 7

¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # dígitos no es spam, promedio # dígitos spam).*

In [16]:
import re

def respuesta_siete():
    spam=spam_data[spam_data['target'] != 0]
    sumLon = 0
    for row in spam.iterrows():
        sumLon += len(re.findall('\d+',row[1].array[0]))
    promSpam = sumLon/len(spam)
    no_spam= spam_data[spam_data['target'] != 1]
    sum1 = 0
    for row in no_spam.iterrows():
        sum1 += len(re.findall('\d+',row[1].array[0]))
    promNoSpam = sum1/len(no_spam)
    return (promNoSpam,promSpam)

In [17]:
respuesta_siete()

(0.2549222797927461, 4.449799196787149)

### Pregunta 8

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y usando **n-gramas de palabras n = 1 a n = 3** (unigramas, bigramas y trigramas).

Usando esta matriz de término-documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* **cantidad de dígitos por documento**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [18]:
from sklearn.linear_model import LogisticRegression

def aux_digit(data):
    longitud = [] 
    for row in data:
        longitud.append(len(re.findall('\d+',row)))
    return np.resize(np.array(longitud),(len(longitud),1))

def respuesta_ocho():
    long = aux(X_train)
    digit = aux_digit(X_train)
    vectorizer = TfidfVectorizer(max_df=0.5,ngram_range = (1,3))
    X_train_vect = vectorizer.fit_transform(X_train)
    X_train_array = X_train_vect.toarray()
    X_train_array = np.concatenate((X_train_array,long,digit),axis =1)
    clf = LogisticRegression(C =100)
    clf.fit(X_train_array,np.array(y_train))
    
    
    long1 = aux(X_test)
    digit1 = aux_digit(X_test)
    X_test_vect = vectorizer.transform(X_test)
    X_test_arra= np.concatenate((X_test_vect.toarray(),long1,digit1), axis=1)
    predictions = clf.predict(X_test_arra)
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [19]:
respuesta_ocho()

/home/pablocg/mineria/lib64/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0 0 0 ... 0 0 1]


0.9670347860041084

### Pregunta 9

¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

*Sugerencia: utilice las clases de caracteres `\ w` y` \ W`*

*Esta función debe devolver una tupla (promedio de # caracteres que no son palabras, no spam, promedio de # caracteres que no son palabras, spam).*

In [20]:
def pregunta_nueve():
    spam=spam_data[spam_data['target'] != 0]
    sumLon = 0
    for row in spam.iterrows():
        sumLon += len(re.findall('\W',row[1].array[0]))
    promSpam = sumLon/len(spam)
    no_spam= spam_data[spam_data['target'] != 1]
    sum1 = 0
    for row in no_spam.iterrows():
        sum1 += len(re.findall('\W',row[1].array[0]))
    promNoSpam = sum1/len(no_spam)
    return (promNoSpam,promSpam)

In [21]:
pregunta_nueve()

(17.29181347150259, 29.041499330655956)

### Pregunta 10

Ajustar y transformar los datos de entrenamiento `X_train` usando un `CountVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y utilizando **caracteres n-grams desde n = 2 a n = 5.**

Para decirle a `CountVectorizer` que use caracteres n-grams, pase en `analyzer = 'char_wb'` que crea caracteres n-gramas solo del texto dentro de los límites de las palabras. Esto debería hacer que el modelo sea más robusto a los errores ortográficos.

Usando esta matriz término documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* cantidad de dígitos por documento
* **cantidad de caracteres que no son palabras (cualquier cosa que no sea una letra, dígito o guión bajo).**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

También **encuentre los 10 coeficientes más pequeños y los 10 más grandes del modelo** y devuélvalos junto con el AUC en una tupla.

La lista de los 10 coeficientes más pequeños debe ordenarse primero, la lista de los 10 coeficientes más grandes debe ordenarse primero.

Las tres características que se agregaron a la matriz de términos del documento deben tener los siguientes nombres si aparecen en la lista de coeficientes:
['longitud_doc', 'conteo_digito', 'caracteres_no_palabra']

*Esta función debe devolver una tupla `(AUC como flotante, lista de coeficientes más pequeños, lista de coeficientes más grande)`.*

In [64]:
def aux_noDigit(data):
    longitud = [] 
    for row in data:
        longitud.append(len(re.findall('\W',row)))
    return np.resize(np.array(longitud),(len(longitud),1))

def coef_list(data):
    noDigit = data[0][data.shape[1]-1]
    digit = data[0][data.shape[1]-2]
    long = data[0][data.shape[1]-3]
    mayores = np.sort(data[0])
    mayores = np.resize(mayores,(1,len(mayores)))[:10]
    menores = np.sort(data[0])[::-1]
    menores = np.resize(menores,(1,len(menores)))[:10]
    mayList = []
    menList = []
    for xs in range(10):
        if mayores[0][xs] == noDigit:
            mayList.append('caracteres_no_palabra')
        elif menores[0][xs] == noDigit:
            menList.append('caracteres_no_palabra')
        elif menores[0][xs] == digit:
            menList.append('conteo_digito')
        elif mayores[0][xs] == digit:
            mayList.append('conteo_digito')
        elif mayores[0][xs] == long:
            mayList.append('longitud_doc')
        elif menores[0][xs] == long:
            menList.append('longitud_doc')
        else:
            mayList.append(mayores[0][xs])
            menList.append(menores[0][xs])
            
    return [mayList, menList]
        

def respuesta_10():
    long = aux(X_train)
    digit = aux_digit(X_train)
    noDigit = aux_noDigit(X_train)
    
    vectorizer = CountVectorizer(max_df=0.5,ngram_range = (2,5), analyzer = 'char_wb')
    X_train_vect = vectorizer.fit_transform(X_train)
    X_train_array = X_train_vect.toarray()
    X_train_array = np.concatenate((X_train_array,long,digit,noDigit),axis =1)
    clf = LogisticRegression(C =100)
    clf.fit(X_train_array,np.array(y_train))
    
    coef = clf.coef_
    coef_result = coef_list(coef)
    
    long1 = aux(X_test)
    digit1 = aux_digit(X_test)
    noDigit1 = aux_noDigit(X_test)
    X_test_vect = vectorizer.transform(X_test)
    X_test_arra= np.concatenate((X_test_vect.toarray(),long1,digit1,noDigit1), axis=1)
    predictions = clf.predict(X_test_arra)
    print(predictions)
    return (roc_auc_score(y_test,predictions),coef_result)


In [65]:
respuesta_10()

/home/pablocg/mineria/lib64/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0 0 0 ... 0 0 1]


(0.9767393002054223,
 [[-0.7106744179473831,
   -0.701436758811417,
   -0.6897274618348326,
   -0.6687712546870493,
   -0.6546083789986065,
   -0.6298854665355934,
   -0.6205310317469349,
   -0.6179826796312491,
   -0.5739241458151942],
  ['conteo_digito',
   1.2189316795990313,
   1.1092778530264071,
   0.7920345629839769,
   0.7699337609976461,
   0.6422208360796439,
   0.6249347269033307,
   0.6222148460708873,
   0.6069824098821436,
   0.6001398194432559]])